In [1]:
import geopandas as gpd

In [2]:
tracts = gpd.read_file("../data/chicago.geojson").to_crs(epsg = 3528)

# https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6
comm   = gpd.read_file("../data/chicago_community_areas.geojson").to_crs(epsg = 3528)
comm["community"] = comm["community"].str.title()
comm["community"] = comm["community"].replace({"Ohare" : "O'Hare"})
comm["caN"] = comm["area_numbe"].astype(int)
comm = comm[["community", "caN", "geometry"]].copy()

chicago_g = comm.unary_union

tracts.set_geometry(tracts.intersection(chicago_g), inplace = True)

In [3]:
tract_ca_map = gpd.sjoin(tracts[["geoid", "geometry"]], 
                         comm[["caN", "geometry"]], how = "inner", op = "intersects").drop("index_right", axis = 1)

tract_ca_map = tract_ca_map.merge(comm.rename(columns = {"geometry" : "ca_geom"}))

tract_ca_map["f"] = tract_ca_map.intersection(tract_ca_map.ca_geom).area / tract_ca_map.area
tract_ca_map.query("f > 0.5", inplace = True)

tract_ca_map = tract_ca_map.set_index("geoid").caN.to_dict()

assert(len(tract_ca_map) == tracts.shape[0])

In [4]:
tracts["caN"] = tracts.geoid.replace(tract_ca_map)

In [5]:
counts = ["den_computers", "n_children", 
          "n_computer", "n_broadband", 
          "n_children_computer_and_broadband", "n_children_no_computer_and_broadband",
          "den_black", "n_black", "den_hispanic", "n_hispanic", "den_ba", "n_ba"]

ca_agg = tracts.groupby("caN")[counts].sum()

ca_agg["f_broadband"] = ca_agg["n_broadband"] / ca_agg["den_computers"]
ca_agg["f_computer"]  = ca_agg["n_computer"]  / ca_agg["den_computers"]

ca_agg["f_children_computer_and_broadband"] = \
    ca_agg["n_children_computer_and_broadband"] / ca_agg["n_children"]


ca_agg["f_ba"]  = ca_agg["n_ba"]  / ca_agg["den_ba"]

ca_agg["f_black"]  = ca_agg["n_black"]  / ca_agg["den_black"]
ca_agg["f_hispanic"]  = ca_agg["n_hispanic"]  / ca_agg["den_hispanic"]

ca_mean = tracts.groupby("caN")[["n100a", "log_mhi"]].mean()

ca_agg = ca_agg.join(ca_mean).reset_index()

ca_agg = comm.merge(ca_agg)

ca_agg.set_geometry(ca_agg.simplify(5), inplace = True)

In [6]:
ca_agg.to_crs(epsg = 4326).to_file("../site/chicago_ca.geojson", driver = "GeoJSON")